In [18]:
# This is the notebook for ReciPete, the recipe-finding chatbot
import re
import numpy
import sqlalchemy
import spacy
import csv
import numpy as np

from nltk.tokenize import word_tokenize

In [312]:
# Set up spacy nlp object
nlp = spacy.load("en_core_web_sm")
words_to_remove = ["and", "then", "with", "in", "of", "but", "for", "or"]

In [129]:
# Create the database object to connect
db_name = 'recipesdb'
db_host = 'localhost'
db_user = 'NeilSinclair'
db_pass = 'Passw0rd!'
db = sqlalchemy.create_engine('mysql+pymysql://{}:{}@{}/{}'.format(db_user, db_pass, db_host, db_name))

In [307]:
#intro_text = "Hi, I'm ReciPete and I'm here to help you find the perfect thing to eat"
with db.connect() as conn:
    ing = conn.execute("SELECT (ingredients) FROM recipes").fetchall()

INGREDIENTS = []
for item in ing:
    temp_ = re.sub('[\(\)\'\\\]', "", str(item)).split(',')[1]
    temp_ = re.sub('[\-\"]', '', temp_).strip()
    if len(temp_) > 0:
        INGREDIENTS.append(temp_)

INGREDIENTS = list(set(INGREDIENTS))

#simplified_ingredients = lambda INGREDIENTS: [item for sublist in INGREDIENTS for item in INGREDIENTS]
SIMP_ING = []
for item in [item.split(' ') for item in INGREDIENTS]:
    for sub in item:
        if sub not in words_to_remove:
            SIMP_ING.append(sub)
SIMP_ING = list(set(SIMP_ING))
#print(SIMP_ING)

In [308]:
with db.connect() as conn:
    ing = conn.execute("SELECT (category) FROM recipes").fetchall()
   
    
CATEGORY = []
for item in ing:
    temp_ = re.sub('[\(\)\'\\\]', "", str(item))
    temp_ = re.sub('[\-\"]', '', temp_).strip()
    if len(temp_) > 0:
        CATEGORY.append(temp_)

CATEGORY = list(set(CATEGORY))

SIMP_CAT = []
for item in [item.split(' ') for item in CATEGORY]:
    for sub in item:
        if sub not in words_to_remove:
            SIMP_CAT.append(sub)
SIMP_CAT = list(set(SIMP_CAT))
#print(SIMP_CAT)

In [ ]:
def parse_request(message):
    # Function that parses what the user wants to eat
    # Example message  - "I'd like something with green peppers" - "I want mexican"
    return 0

In [51]:
test_msg1 = "I'd like something with green peppers in it"
test_msg2 = "I feel like mexican tonight"
test_msg3 = "I don't want to eat pasta! Nor pizza, apples, pies - or even meat"
test_msg4 = "sweet potatoes and 10 tomotoes"

doc1 = nlp(test_msg1)
doc2 = nlp(test_msg2)
word_tokenize(test_msg4)

['sweet', 'potatoes', 'and', '10', 'tomotoes']

In [124]:
message = "I want something with peppers and cheese, but I don't want mushrooms or anchovies"
message2 = "I want something that cooks quickly, but without salt and "
ingredients = ["peppers", "cheese", "feta cheese", "apples", "potatoes", "sweet potatoes", "anchovies", "mushrooms"]
food_categories = ["asian", "african", "carribean"]
negative_sentiment = ["dont", "not", "without", "hate", "dislike", "cant", "wont", "without"]

In [125]:
def parse_pos_neg(message, ingredients, neg_sentiment):
    # Function that takes a message and returns a dictionary of positive and negative elements
    # Parameters - message - the user's message
    #            - ingredients - the ingredients and categories identified in the message
    #            - neg_sentiment - a list with words indicating negative sentiment
    # Find the first entity (ingredient) and find whether it's positive or not
    tokens = message.split(' ')
    tokens = [re.sub('[,!\-\']', '', token.lower()) for token in tokens]
    
    # Take each ingredient token and find it's place in the sentence
    ingredient_tracker = {}
    for n in range(len(tokens)):
        if tokens[n] in ingredients:
            ingredient_tracker[tokens[n]] = n
     
    # Find the sentiment
    ingredient_sentiment = {}
    previous_sentiment = None
    last_ingr = 0
    conjunctions = ["and", "or", "either", "nor"]
    
    for key, val in ingredient_tracker.items():
        obj_sentiment = tokens[last_ingr:val+1]
        
        # First check if the first word in the object_sentiment string is a conjuction as that would indicate that it has
        # the same sentiment as the previous ingredient
        if obj_sentiment[0] in conjunctions:
            ingredient_sentiment[key] = previous_sentiment
        
        elif any(item in obj_sentiment for item in neg_sentiment):
            ingredient_sentiment[key] = "negative"
            previous_sentiment = "negative"
        else:
            ingredient_sentiment[key] = "postive"       
            previous_sentiment = "postive"
        last_ingr = val+1
             
    return ingredient_sentiment

print(parse_pos_neg(message, ingredients, negative_sentiment))


{'peppers': 'postive', 'cheese': 'postive', 'mushrooms': 'negative', 'anchovies': 'negative'}


In [309]:
def take_action(message):
    # Function that takes in a message, parses it for positives and negatives and takes action
    # Returns - the action requested by the user
    
    # Extract the ingredients and/or categories from the message, with SIMP_ING and SIMP_CAT as global lists
    tokens = message.split(' ')
    tokens = [re.sub('[,!\-\']', '', token.lower()) for token in tokens]
    ingredients = [token for token in tokens if token in SIMP_ING]
    
    return ingredients

In [311]:
a = take_action("I want something with potatoes and beef but not anchovies or pork")
print(a)

['potatoes', 'beef', 'or', 'pork']
